In [1]:
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation, Flatten, Conv2D, MaxPooling2D,BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import os 




/home/thanawit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/thanawit/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#initialte the variable

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation  = True
num_predictions = 20 
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
#downloading the dataset

(x_train , y_train) , (x_test , y_test ) = cifar10.load_data()

In [4]:
#convert class to binary class

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test =  keras.utils.to_categorical(y_test, num_classes)

In [5]:
x_train.shape[1:]

(32, 32, 3)

In [6]:
#Initialize Layer

model = Sequential()
#Conv2D Filter = filter size, kernel size, stride , padding , format , dilation rate , activation , biased, initializer, bias initilizaer, more 

#Block 1 
model.add(Conv2D( 32 , (3,3) , padding='same',input_shape = x_train.shape[1:]  ))
# 2. Using ReLU instead of Sigmoid
model.add(Activation('relu'))
# 5. Using BatchNorm to Increase Accuracy
model.add(BatchNormalization())
model.add(Conv2D(32 , (3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
# 3. Using Dropout to Increase Accura
model.add(Dropout(0.3))

#Block 2 
model.add(Conv2D(64, (3,3) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64 , (3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#Connecting it to Classification
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))



In [7]:
# 6. Scheduling LR decaying 

def lr_schedule(epoch):
    
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 150:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [8]:
#initiate training lost
#4. Second parameter is decayed 
opt = keras.optimizers.rmsprop(lr_schedule(0), decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#meaning all the result
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

Learning rate:  0.001


In [9]:
#Subtract mean to make it more normalize
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

In [10]:
#Decaying LR

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [ lr_reducer, lr_scheduler]


#train model 
# model.fit(x_train , y_train, batch_size=batch_size , epochs= epochs , validation_data=(x_test, y_test), shuffle=True , callbacks=callbacks)

model.fit(x_train , y_train, batch_size=batch_size , epochs= epochs , validation_data=(x_test, y_test), shuffle=True )
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)



Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 13s - loss: 1.5007 - acc: 0.4933 - val_loss: 1.1968 - val_acc: 0.5985
Epoch 2/100
50000/50000 [==============================] - 12s - loss: 1.0183 - acc: 0.6464 - val_loss: 0.8722 - val_acc: 0.6991
Epoch 3/100
50000/50000 [==============================] - 12s - loss: 0.8539 - acc: 0.7052 - val_loss: 0.9177 - val_acc: 0.6866
Epoch 4/100
50000/50000 [==============================] - 12s - loss: 0.7728 - acc: 0.7361 - val_loss: 0.7685 - val_acc: 0.7404
Epoch 5/100
50000/50000 [==============================] - 12s - loss: 0.7114 - acc: 0.7561 - val_loss: 0.7498 - val_acc: 0.7455
Epoch 6/100
50000/50000 [==============================] - 12s - loss: 0.6584 - acc: 0.7758 - val_loss: 0.6946 - val_acc: 0.7695
Epoch 7/100
50000/50000 [==============================] - 12s - loss: 0.6209 - acc: 0.7879 - val_loss: 0.7751 - val_acc: 0.7356
Epoch 8/100
50000/50000 [======================

In [11]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

 9760/10000 [============================>.] - ETA: 0sTest loss: 0.798695784664154
Test accuracy: 0.8155
